In [1]:
from __future__ import print_function

from models.ResNet_3D.model import generate_model
from models.ResNet_3D.opts import create_args
from models.ResNet_3D.mean import get_mean, get_std

import torch

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scripts.read_video import generate_block
import time
import os
import argparse

In [2]:
def find_length(filename, seg_length=1):
    """
    Function to find the length of the video in terms of specified segments.
    """
    return sum(1 for _ in generate_block(filename, 1))

In [15]:
def get_file_list(filename):
    with open(filename, 'r') as file:
        f = file.read()
    return f.split('\n')[:-1]

In [4]:
def generate_features(resnet, filename):
    """
    Function to process the current video file and generate the c3d fetures.
    
    Parameters
    resnet          :torch.nn.Module
                     The c3d feature extraction model.
    filename        :str
                     The filename of the videos to return the features
    
    Returns
    -------
    features        :numpy.array
                     The features of the video in the required shape.
    """
    total_length = find_length(filename)
    block = generate_block(filename, 1)
    start_time = time.time()
    feature_arr = []
    for i, curr_block in enumerate(block):
        print("{}/{}".format(i+1, total_length), sep='\r', end='\r')
        features = resnet(curr_block['block'])
        features = (features - features.mean())/(features.max() - features.mean())
        feature_arr.append(features.cpu().detach().numpy())
        del features
    print("")
    total_time = time.time()-start_time
    features = np.rollaxis(np.array(feature_arr), 1 )[0]
    return features

In [5]:
def iterate_folder(base_path, c3d):
    """
    Function to iterate over the video files in the given folder
    and call the function to generate the c3d features and write the
    features in a csv file.
    Parameters
    ----------
    base_path       :str
                     Path of the folder containing the video files.
    resnet          :torch.nn.Module
                     The resnet feature extraction model.
    Returns
    -------
    None
    """
    start_time = time.time()
    total_files = len(os.listdir(base_path))
    category = base_path.split('/')[-1]
    print("Calculating features for :", category)

    for i, vid_file in enumerate(os.listdir(base_path)):
        features = generate_features(c3d, os.path.join(base_path, vid_file))
        mult = len(features)//32
        feature_norm_arr = []
        for i in range(32):
            sub_arr = np.linalg.norm(features[i*mult:(i+1)*mult], axis=0)
            feature_norm_arr.append(sub_arr)
        folder_path = os.path.join("results", "features", category)
        file_path = os.path.join(folder_path, vid_file.split('.')[0]+'.csv')
        if not(os.path.exists(folder_path)):
            os.mkdir(folder_path)
        pd.DataFrame(np.array(feature_norm_arr, dtype='float16'), index=None).to_csv(file_path, header=False)
                     
        print("{}/{} completed".format(i+1, total_files), sep='\r', end='\r')

In [17]:
def iterate_list(file_list, c3d):
    """
    Function to iterate over the video files in the given folder
    and call the function to generate the c3d features and write the
    features in a csv file.
    Parameters
    ----------
    base_path       :str
                     Path of the folder containing the video files.
    resnet          :torch.nn.Module
                     The resnet feature extraction model.
    Returns
    -------
    None
    """
    start_time = time.time()
    category = base_path.split('/')[-1]
    total_files = len(file_list)
    print("Calculating features for :", category)

    for i, vid_file in enumerate(file_list):
        vid_file = vid_file.split('/')[-1]
        features = generate_features(c3d, os.path.join(base_path, vid_file))
        mult = len(features)//32
        feature_norm_arr = []
        for i in range(32):
            sub_arr = np.linalg.norm(features[i*mult:(i+1)*mult], axis=0)
            feature_norm_arr.append(sub_arr)
        folder_path = os.path.join("results", "features", category)
        file_path = os.path.join(folder_path, vid_file.split('.')[0]+'.csv')
        if not(os.path.exists(folder_path)):
            os.mkdir(folder_path)
        pd.DataFrame(np.array(feature_norm_arr, dtype='float16'), index=None).to_csv(file_path, header=False)
                     
        print("{}/{} completed".format(i+1, total_files), sep='\r', end='\r')

In [12]:
opt = create_args(model_name = 'resnext', model_depth=101, \
                  model='/home/ubuntu/Downloads/PretrainedModels/resnext-101-kinetics.pth')
opt.mean = get_mean()
opt.std = get_std()
opt.arch = '{}-{}'.format(opt.model_name, opt.model_depth)
opt.sample_size = 112
opt.sample_duration = 16
opt.n_classes = 400

In [13]:
model = generate_model(opt)
print('loading model {}'.format(opt.model))
model_data = torch.load(opt.model)
assert opt.arch == model_data['arch']
model.load_state_dict(model_data['state_dict'],strict=False)
model = model.eval()

loading model /home/ubuntu/Downloads/PretrainedModels/resnext-101-kinetics.pth


In [9]:
base_path = '/home/ubuntu/Project/UCF_Crimes/Videos/RoadAccidents'
iterate_folder(base_path, model)

Calculating features for : RoadAccidents
76/76
119/119completed
107/107completed
83/831 completed
34/341 completed
6/6191 completed
78/781 completed
36/361 completed
109/109completed
37/371 completed
55/551 completed
21/211 completed
42/421 completed
100/100completed
73/731 completed
142/142completed
31/311 completed
128/128completed
70/701 completed
88/881 completed
126/126completed
349/349completed
169/169completed
65/651 completed
206/206completed
70/701 completed
54/541 completed
25/251 completed
140/140completed
101/101completed
51/511 completed
127/127completed
126/126completed
80/801 completed
154/154completed
79/791 completed
31/311 completed
39/391 completed
64/641 completed
206/206completed
105/105completed
75/751 completed
87/871 completed
33/331 completed
26/261 completed
20/201 completed
185/185completed
40/401 completed
270/270completed
79/791 completed
127/127completed
129/129completed
53/531 completed
73/731 completed
35/351 completed
31/311 completed
236/236completed
6

In [18]:
base_path = '/home/ubuntu/Project/UCF_Crimes/Videos/Training_Normal_Videos_Anomaly'
file_list = get_file_list('/home/ubuntu/Project/UCF_Crimes/list_normal_road.txt')
iterate_list(file_list, model)

Calculating features for : Training_Normal_Videos_Anomaly
22/22
71/711 completed
174/174completed
71/711 completed
22/221 completed
51/511 completed
77/771 completed
147/147completed
23/231 completed
14/141 completed
180/180completed
92/921 completed
91/911 completed
77/771 completed
75/751 completed
95/951 completed
70/701 completed
1162/1162mpleted
61/611 completed
279/279completed
184/184completed
105/105completed
266/266completed
55/551 completed
6249/6249mpleted
6288/6288mpleted
723/723completed
43/431 completed
63/631 completed
60/601 completed
98/981 completed
113/113completed
528/528completed
131/131completed
218/218completed
146/146completed
48/481 completed
201/201completed
28/281 completed
111/111completed
82/821 completed
178/178completed
1369/1369mpleted
192/192completed
74/741 completed
250/250completed
55/551 completed
49/491 completed
196/196completed
106/106completed
106/106completed
169/169completed
149/149completed
107/107completed
6346/6346mpleted
56/561 completed
1